# LSTM-GloVe Model for Sentiment Analysis in text data

En este notebook se describe el proceso de carga, preprocesamiento, embedding, construcción y entrenamiento de un modelo que emplea LSTM y GloVe
para el set de datos de texto.

## Carga de Datos

A continuación se cargará el dataset unificado que se construyó en etapas anteriores (Ver `data_join.ipynb` y `NLP_tasks.ipynb`) que cuenta con cas 66000 registros de texto.

In [1]:
import pandas as pd
data = pd.read_csv('../../data/cleaned/out.csv')
data.head()

,label,text
0,0,@tiffanylue i know i was listenin to bad habi...
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
3,1,wants to hang out with friends SOON!
4,6,@dannycastillo We want to trade with someone w...


In [2]:
data.shape

(65989, 2)

## Preprocesamiento

### Emociones

Se realizará el método de one_hot_encoding para nuestra varible de salida del modelo: Las 7 emociones.

In [3]:
#Emociones
emotions = data['label'].unique()
print(emotions)

[0 1 6 4 5 2 3]


In [4]:
y = pd.get_dummies(data.label)
y

,0,1,2,3,4,5,6
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
65984,0,0,0,0,0,0,1
65985,0,0,0,0,0,0,1
65986,0,0,0,0,0,1,0
65987,0,0,0,0,0,0,1


### Texto

Es necesario remover de nuestros datos información irrelevante como etiquetas, puntución, números y caracteres especiales.

In [5]:
import re
data['text'][0]

'@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =['

In [6]:
TAG_RE = re.compile(r'@[^> ]+')

def remove_at_sign(sentence: str):
    '''
    Replaces '@' from and input string for an empty space
    :param sentence: String that contains @
    :return: sentence without @
    '''

    return TAG_RE.sub('', sentence)

In [7]:
remove_at_sign(data['text'][0])

' i know  i was listenin to bad habit earlier and i started freakin at his part =['

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yonosoysantiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords

def preprocess_text(sentence: str):
    '''
    Cleans up a sentence leaving only 2 or more non-stopsentences composed of upper and lowercase
    :param sentence: String to be cleaned
    :return: sentence without numbers, special chars and long stopsentences
    '''

    cleaned_sentence = sentence.lower()
    cleaned_sentence = remove_at_sign(cleaned_sentence)
    cleaned_sentence = re.sub('[^a-zA-Z]', ' ', cleaned_sentence)
    cleaned_sentence = re.sub('\s+[a-zA-Z]\s', ' ', cleaned_sentence)
    cleaned_sentence = re.sub('\s+', ' ', cleaned_sentence)

    #Removal of stopsentences
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s')
    cleaned_sentence = pattern.sub('', cleaned_sentence)

    return cleaned_sentence

In [10]:
preprocess_text(data['text'][0])

' know listenin bad habit earlier started freakin part '

In [11]:
from copy import  deepcopy

cleaned_data = deepcopy(data)
cleaned_data['text'] = cleaned_data['text'].apply(preprocess_text)
cleaned_data['text']

0         know listenin bad habit earlier started freak...
1                   layin bed headache ughhhh waitin call 
2                          funeral ceremony gloomy friday 
3                                 wants hang friends soon 
4                  want trade someone houston tickets one 
                               ...                        
65984                                                     
65985                       got uh joey women adam apples 
65986                                  guys messing right 
65987                                                yeah 
65988                           good one second like whoa 
Name: text, Length: 65989, dtype: object

## Embedding

Para el proceso de embedding se usarán los datos de un modelo de embedding como lo es GloVe.
La información dicho modelo será cargada dentro de `words`. Cada uno de los tokens de GloVe que
se usará tiene una dimensión de 50. En caso de que

In [12]:
import numpy as np

words = {}

def add_to_dict(dictionary, filename):
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.split(' ')

            try:
                dictionary[line[0]] = np.array(line[1:], dtype=float)
            except:
                continue

add_to_dict(words, './GloVe/glove.6B/glove.6B.50d.txt')

In [13]:
# words

### Tokenización y Lematización

Una vez cargada la información de los tokens de GloVe se procede a tokenizar y lematizar cada
una de las oraciones en nuestro set de datos.

In [14]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/yonosoysantiago/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/yonosoysantiago/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Ejemplo de cómo se debería de tokenizar y lematizar una oración:

In [15]:
from nltk import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

sample = preprocess_text(cleaned_data['text'][0])
token_sample = tokenizer.tokenize(sample)
lemma_sample = [lemmatizer.lemmatize(token) for token in token_sample]
lemma_sample

['know', 'listenin', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part']

A continuación se define una función para la tokenización y lematización. Adicionalmente, el token final que se entrega únicamente contiene palabras definidas en `words`.

In [16]:
def sentence_to_token_list(sentence: str):
    tokens = tokenizer.tokenize(sentence)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    useful_tokens = [token for token in lemmatized_tokens if token in words]

    return  useful_tokens

In [17]:
sentence_to_token_list(sample)

['know', 'bad', 'habit', 'earlier', 'started', 'freakin', 'part']

Con el token anterior, el cual sabemos se puede representar por medio de uno de los tokens almacenados en `words`, entonces pasamos a la representación de estos:

In [18]:
def sentence_to_words_vectors(sentence: str, word_dict=words):
    processed_tokens = sentence_to_token_list(sentence)

    vectors = []
    for token in processed_tokens:
        if token in word_dict:
            token_vector = word_dict[token]
            vectors.append(token_vector)

    return np.array(vectors, dtype=float)

In [19]:
sentence_to_words_vectors(sample).shape

(7, 50)

In [20]:
sentence_to_words_vectors(sample)

array([[ 0.28328  ,  0.12887  ,  0.29861  , -0.23238  ,  0.88263  ,
        -0.30814  , -0.61148  ,  0.37254  , -0.40449  ,  0.17351  ,
        -0.13958  ,  0.67868  , -0.55673  , -0.13453  ,  1.1441   ,
         0.53457  ,  0.59148  ,  0.19083  ,  0.25406  , -0.71581  ,
        -0.87951  ,  0.72879  ,  0.67054  ,  0.62866  ,  1.0034   ,
        -2.2327   , -1.194    , -0.06245  ,  0.93933  , -1.1132   ,
         2.926    ,  0.59809  , -0.51255  , -0.49886  , -0.016524 ,
        -0.20697  , -0.059229 , -0.0072689,  0.53314  , -0.14621  ,
        -0.21417  ,  0.24102  ,  0.21648  ,  0.73351  ,  0.43582  ,
         0.045278 , -0.25489  ,  0.15273  , -0.13088  ,  0.89397  ],
       [-0.17981  , -0.40407  , -0.1653   , -0.60687  , -0.39656  ,
         0.12688  , -0.053049 ,  0.38024  , -0.51008  ,  0.46593  ,
        -0.30818  ,  0.79362  , -0.85766  , -0.25143  ,  1.0448   ,
         0.18628  ,  0.13688  ,  0.092588 , -0.2236   , -0.13604  ,
        -0.19482  ,  0.057702 ,  0.56133  ,  0.

In [21]:
#Se obtiene nuestro conjunto de datos X
X = cleaned_data['text'].apply(lambda sentence: sentence_to_words_vectors(sentence))
X

0        [[0.28328, 0.12887, 0.29861, -0.23238, 0.88263...
1        [[0.062774, 0.25423, 0.54447, -0.49013, -0.488...
2        [[0.89902, 1.5889, -0.34232, -0.40286, 0.97148...
3        [[0.13627, -0.054478, 0.3703, -0.41574, 0.6056...
4        [[0.13627, -0.054478, 0.3703, -0.41574, 0.6056...
                               ...                        
65984                                                   []
65985    [[-0.4097, -0.37167, 0.38852, -0.34947, 0.5425...
65986    [[-0.76156, -0.012958, -0.14249, -0.5142, 1.50...
65987    [[-0.80924, -0.030977, 0.5102, -0.75298, 0.490...
65988    [[-0.35586, 0.5213, -0.6107, -0.30131, 0.94862...
Name: text, Length: 65989, dtype: object

Dado que las matrices de vectores de cada oración tienen un número diferente de filas debido a que cada oración cuenta con un número diferente de palabas. Es necesario identificar el tamaño máximo de los textos que se tienen para su "estandarización":

In [22]:
temporal = deepcopy(X)
temporal['len'] = temporal.apply(np.shape)

MAX_LEN = max(temporal['len'])[0]
MAX_LEN

35

Dado que el tamaño máximo es 35, entonces se llevarán todas las matrices a la forma `(35, 50)`. Los valores faltantes para cada vector serán 0s en su inicio.

In [23]:
import tensorflow as tf

X_ = tf.keras.utils.pad_sequences(X, maxlen=MAX_LEN, dtype='float32')
# X_

2023-05-01 20:53:42.673625: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 20:53:44.945779: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/yonosoysantiago/miniconda3/envs/tf211_conda/lib/
2023-05-01 20:53:44.946258: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/yonosoysantiago/miniconda3/envs/tf211_conda/lib/
2023-05-01 20:53:4

In [24]:
X_.shape

(65989, 35, 50)

In [25]:
X_[0].shape

(35, 50)

In [26]:
X_[0]

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [-0.20492 , -0.47264 , -0.24182 , ..., -0.58691 , -0.31341 ,
        -0.20058 ],
       [-0.081298, -0.18447 , -0.20424 , ..., -0.027863,  0.64354 ,
         0.59983 ],
       [ 0.70504 ,  0.18255 , -0.75188 , ..., -0.13137 , -0.41208 ,
        -0.092147]], dtype=float32)

## División Entrenamiento-Validación-Test

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X_, y, test_size=0.2, random_state=3)

# Modelos

In [58]:
from keras.models import Sequential
from keras import layers
from keras.layers import Embedding, Lambda, LSTM, Flatten, Dense, Input, Dropout, Bidirectional, GlobalMaxPooling1D
from keras.optimizers import Adam, RMSprop, SGD
from kerastuner import RandomSearch, HyperParameters
from keras.layers import Conv1D, MaxPooling1D

def print_hyperparameters(json_data):
    values = json_data["values"]
    
    for key, value in sorted(values.items()):
        print(f"{key}: {value}")

In [29]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-5)
cp = ModelCheckpoint('saved/', save_best_only=True)

callbacks = [cp, early_stopping, reduce_lr]

## GloVe-LSTM

In [30]:


def build_model(hp):
    
    model = Sequential()
    model.add(Input(shape=(MAX_LEN, 50)))

    # Hiperparámetros para LSTM 1
    lstm_units = hp.Int("lstm_units_1", min_value=128, max_value=256, step=32)
    lstm_dropout = hp.Float("lstm_dropout", min_value=0.1, max_value=0.5, step=0.1)
    
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=lstm_dropout, recurrent_dropout=lstm_dropout)))

    # Hiperparámetros para LSTM 2
    lstm_units = hp.Int("lstm_units_2", min_value=64, max_value=128, step=32)
    lstm_dropout = hp.Float("lstm_dropout_2", min_value=0.1, max_value=0.5, step=0.1)
    
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=lstm_dropout, recurrent_dropout=lstm_dropout)))
    model.add(GlobalMaxPooling1D())

    # Hiperparámetros para capa densa 1
    dense_units = hp.Int("dense_units_1", min_value=32, max_value=128, step=32)
    dense_dropout = hp.Float("dense_dropout_1", min_value=0.1, max_value=0.5, step=0.1)
    
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dense_dropout))

    # Hiperparámetros para capa densa 2
    dense_units = hp.Int("dense_units_2", min_value=32, max_value=128, step=32)
    model.add(Dense(dense_units, activation='relu'))

    # Salida del modelo
    model.add(Dense(7, activation='softmax'))

    # Hiperparámetros para el optimizador (En otras pruebas se vio que Adam era el mejor)
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="LOG")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define el objeto de búsqueda aleatoria
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=20,  # Número de modelos a probar
    executions_per_trial=1,
    directory='./saved/fine_tuned/',
    project_name='HP_LSTM_Glove_text'
)

# Resumen de la búsqueda
tuner.search_space_summary()

INFO:tensorflow:Reloading Tuner from ./saved/fine_tuned/HP_LSTM_Glove_text/tuner0.json
Search space summary
Default search space size: 8
lstm_units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
lstm_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
lstm_units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
lstm_dropout_2 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dense_dropout_1 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 

In [31]:
BATCH_SIZE=1024
tuner.search(X_train, y_train,
                    epochs=10,
                    validation_split=0.1,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)



INFO:tensorflow:Oracle triggered exit


In [32]:
best_hp_gloveLstm = tuner.get_best_hyperparameters()[0]

print("Mejores hiperparámetros encontrados:")
print(best_hp_gloveLstm.get_config())


Mejores hiperparámetros encontrados:
{'space': [{'class_name': 'Int', 'config': {'name': 'lstm_units_1', 'default': None, 'conditions': [], 'min_value': 128, 'max_value': 256, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'lstm_dropout', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'lstm_units_2', 'default': None, 'conditions': [], 'min_value': 64, 'max_value': 128, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'lstm_dropout_2', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'dense_units_1', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'dense_dropout_1', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 

In [33]:
# from keras.models import Sequential
# from keras.layers import Embedding, LSTM, Flatten, Dense, Input, Dropout, Bidirectional, GlobalMaxPooling1D

# model = Sequential()
# model.add(Input(shape=(MAX_LEN, 50)))
# model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.25, recurrent_dropout=0.25)))
# model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.25, recurrent_dropout=0.25)))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(7, activation='softmax'))

## Test

In [34]:
from keras.models import load_model

lstm_basic_model = load_model('saved/')

2023-05-01 20:53:47.812328: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 20:53:47.812540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 20:53:47.926150: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 20:53:47.926649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-01 20:53:47.927015: I tensorflow/compiler/xla/stream_executo

In [35]:
score = lstm_basic_model.evaluate(X_test, y_test, verbose=1)

2023-05-01 20:53:55.191498: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100


 43/413 [==>...........................] - ETA: 1s - loss: 1.8443 - accuracy: 0.2493

2023-05-01 20:53:56.502317: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


413/413 [==============================] - 4s 3ms/step - loss: 1.8402 - accuracy: 0.2466


In [36]:
print('Test Accuracy:', score[1])

Test Accuracy: 0.2466282844543457


## GloVe-LSTM-DO

In [37]:
lstm_dropout_dense = Sequential(name='Lstm-dout-dense')
lstm_dropout_dense.add(Input(shape=(MAX_LEN, 50)))
lstm_dropout_dense.add(LSTM(64, return_sequences=True))
lstm_dropout_dense.add(Dropout(0.2))
lstm_dropout_dense.add(LSTM(32))
lstm_dropout_dense.add(Flatten())
lstm_dropout_dense.add(Dense(128, activation='relu'))
lstm_dropout_dense.add(Dense(7, activation='softmax'))

In [38]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, Flatten, Dense
from keras.optimizers import Adam, RMSprop, SGD
from kerastuner import RandomSearch, HyperParameters

def build_model_gloveLstmDo(hp):
    model = Sequential(name='GloVe-LSTM-DO')
    model.add(Input(shape=(MAX_LEN, 50)))
    
    # Hiperparámetros para la primera capa LSTM
    lstm_units_1 = hp.Int("lstm_units_1", min_value=32, max_value=128, step=32)
    model.add(LSTM(lstm_units_1, return_sequences=True))
    
    # Hiperparámetros para Dropout
    dropout_rate = hp.Float("dropout_rate", min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(dropout_rate))
    
    # Hiperparámetros para la segunda capa LSTM
    lstm_units_2 = hp.Int("lstm_units_2", min_value=16, max_value=64, step=16)
    model.add(LSTM(lstm_units_2))
    
    model.add(Flatten())
    
    # Hiperparámetros para la capa Densa
    dense_units = hp.Int("dense_units", min_value=32, max_value=256, step=32)
    model.add(Dense(dense_units, activation='relu'))
    
    model.add(Dense(7, activation='softmax'))

    # Hiperparámetros para el optimizador
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="LOG")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

gloveLstmDoTuner = RandomSearch(
    build_model_gloveLstmDo,
    objective="val_accuracy",
    max_trials=20,  # Número de modelos a probar
    executions_per_trial=1,
    directory='./saved/fine_tuned/',
    project_name='HP_LSTM_dropout_dense'
)

# Resumen de la búsqueda
gloveLstmDoTuner.search_space_summary()

INFO:tensorflow:Reloading Tuner from ./saved/fine_tuned/HP_LSTM_dropout_dense/tuner0.json
Search space summary
Default search space size: 5
lstm_units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
lstm_units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [39]:
# Realiza la búsqueda
gloveLstmDoTuner.search(X_train, y_train,
             epochs=10,
             validation_split=0.1,
             batch_size=4096,
             callbacks=callbacks)

best_hp_gloveLstmDo = gloveLstmDoTuner.get_best_hyperparameters(num_trials=1)[0]

INFO:tensorflow:Oracle triggered exit


In [40]:
# lstm_dropout_dense.summary()

In [41]:
# lstm_dropout_dense.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['acc'])

In [42]:
# early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [43]:
# history_2 = lstm_dropout_dense.fit(X_train, y_train,
#                     validation_split=0.2, epochs=10,
#                     batch_size=BATCH_SIZE,
#                     callbacks=[early_stop])

In [44]:
# score = lstm_dropout_dense.evaluate(X_test, y_test, verbose=1)
# print('Test Accuracy:', score[1])

# 3LSTM-DO-CNN-Dense

In [45]:
# from keras.layers import Conv1D, MaxPooling1D
# from keras.models import Sequential
# from keras.layers import Input, LSTM, Dropout, Flatten, Dense
# from keras.optimizers import Adam, RMSprop, SGD
# from kerastuner import RandomSearch, HyperParameters

# lstm3_do_cnn_dense = Sequential(name='3LSTM-DO-CNN-Dense')
# lstm3_do_cnn_dense.add(Input(shape=(MAX_LEN, 50)))

# #LSTM
# lstm3_do_cnn_dense.add(LSTM(256, name='LSTM1', return_sequences=True))
# lstm3_do_cnn_dense.add(Dropout(0.2, name='DO1'))

# lstm3_do_cnn_dense.add(LSTM(128, name='LSTM2', return_sequences=True))
# lstm3_do_cnn_dense.add(Dropout(0.2, name='DO2'))

# lstm3_do_cnn_dense.add(LSTM(64, name='LSTM3', return_sequences=True))
# lstm3_do_cnn_dense.add(Dropout(0.2, name='DO3'))

# #CNN
# lstm3_do_cnn_dense.add(Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
# lstm3_do_cnn_dense.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
# lstm3_do_cnn_dense.add(Dropout(0.2))
# lstm3_do_cnn_dense.add(Flatten(name='F1'))

# #Fully connected
# lstm3_do_cnn_dense.add(Dense(64, activation='relu'))
# lstm3_do_cnn_dense.add(Dense(7, activation='softmax'))

# lstm3_do_cnn_dense.summary()

In [46]:
def build_model_gloveLstmDoCnn(hp):
    model = Sequential(name='3LSTM-DO-CNN-Dense')
    model.add(Input(shape=(MAX_LEN, 50)))

    # Hiperparámetros para la primera capa LSTM
    lstm_units_1 = hp.Int("lstm_units_1", min_value=128, max_value=512, step=32)
    model.add(LSTM(lstm_units_1, name='LSTM1', return_sequences=True))
    lstm_dropout_1 = hp.Float("lstm_dropout_1", min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(lstm_dropout_1, name='DO1'))

    # Hiperparámetros para la segunda capa LSTM
    lstm_units_2 = hp.Int("lstm_units_2", min_value=64, max_value=256, step=32)
    model.add(LSTM(lstm_units_2, name='LSTM2', return_sequences=True))
    lstm_dropout_2 = hp.Float("lstm_dropout_2", min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(lstm_dropout_2, name='DO2'))

    # Hiperparámetros para la tercera capa LSTM
    lstm_units_3 = hp.Int("lstm_units_3", min_value=32, max_value=128, step=16)
    model.add(LSTM(lstm_units_3, name='LSTM3', return_sequences=True))
    lstm_dropout_3 = hp.Float("lstm_dropout_3", min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(lstm_dropout_3, name='DO3'))

    # Hiperparámetros para la capa Conv1D
    conv_filters = hp.Int("conv_filters", min_value=32, max_value=256, step=32)
    model.add(Conv1D(conv_filters, kernel_size=3, strides=1, padding='same', activation='relu'))
    
    model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))

    cnn_dropout = hp.Float("cnn_dropout", min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(cnn_dropout))
    
    model.add(Flatten(name='F1'))

    # Hiperparámetros para la capa Densa
    dense_units = hp.Int("dense_units", min_value=32, max_value=256, step=32)
    model.add(Dense(dense_units, activation='relu'))
    
    model.add(Dense(7, activation='softmax'))

    # Hiperparámetros para el optimizador
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="LOG")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [47]:
tuner = RandomSearch(
    build_model_gloveLstmDoCnn,
    objective="val_accuracy",
    max_trials=20,  # Número de modelos a probar
    executions_per_trial=1,
    directory='./saved/fine_tuned/',
    project_name='HP_3LSTM-DO-CNN-Dense'
)

# Resumen de la búsqueda
tuner.search_space_summary()

INFO:tensorflow:Reloading Tuner from ./saved/fine_tuned/HP_3LSTM-DO-CNN-Dense/tuner0.json
Search space summary
Default search space size: 10
lstm_units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
lstm_dropout_1 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
lstm_units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
lstm_dropout_2 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
lstm_units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
lstm_dropout_3 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
conv_filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 3

In [48]:
BATCH_SIZE=4096
tuner.search(X_train, y_train,
                    epochs=10,
                    validation_split=0.1,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)

best_hp_gloveLstmDoCnn = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Mejores hiperparámetros encontrados:")
print_hyperparameters(best_hp_gloveLstmDoCnn.get_config())

INFO:tensorflow:Oracle triggered exit
Mejores hiperparámetros encontrados:
cnn_dropout: 0.5
conv_filters: 256
dense_units: 224
learning_rate: 0.0009359146486726661
lstm_dropout_1: 0.30000000000000004
lstm_dropout_2: 0.30000000000000004
lstm_dropout_3: 0.4
lstm_units_1: 384
lstm_units_2: 256
lstm_units_3: 32


# Test

In [49]:
print("MODELO 1")
print_hyperparameters(best_hp_gloveLstm.get_config())
print(" ==================================================== ")
print("MODELO 2")
print_hyperparameters(best_hp_gloveLstmDo.get_config())
print(" ==================================================== ")
print("MODELO 3")
print_hyperparameters(best_hp_gloveLstmDoCnn.get_config())

MODELO 1
dense_dropout_1: 0.30000000000000004
dense_units_1: 128
dense_units_2: 96
learning_rate: 0.0005820326357694493
lstm_dropout: 0.30000000000000004
lstm_dropout_2: 0.4
lstm_units_1: 224
lstm_units_2: 96
MODELO 2
dense_units: 128
dropout_rate: 0.1
learning_rate: 0.0008080792741293362
lstm_units_1: 128
lstm_units_2: 32
MODELO 3
cnn_dropout: 0.5
conv_filters: 256
dense_units: 224
learning_rate: 0.0009359146486726661
lstm_dropout_1: 0.30000000000000004
lstm_dropout_2: 0.30000000000000004
lstm_dropout_3: 0.4
lstm_units_1: 384
lstm_units_2: 256
lstm_units_3: 32


In [50]:
def best_model_1(hyper_parameters):
    model = Sequential()
    model.add(Input(shape=(MAX_LEN, 50)))
    
    model.add(Bidirectional(LSTM(hyper_parameters.get('lstm_units_1'), 
                                 return_sequences=True, 
                                 dropout=hyper_parameters.get('lstm_dropout'), 
                                 recurrent_dropout=hyper_parameters.get('lstm_dropout'))))
    
    model.add(Bidirectional(LSTM(hyper_parameters.get('lstm_units_2'), 
                                 return_sequences=True, 
                                 dropout=hyper_parameters.get('lstm_dropout_2'), 
                                 recurrent_dropout=hyper_parameters.get('lstm_dropout_2'))))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(hyper_parameters.get('dense_units_1'), activation='relu'))
    model.add(Dropout(hyper_parameters.get('dense_dropout_1')))
    model.add(Dense(hyper_parameters.get('dense_units_2'), activation='relu'))
    model.add(Dense(7, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [51]:
best_model_gloveLstm = best_model_1(best_hp_gloveLstm)
best_model_gloveLstm.fit(X_train, y_train,
             epochs=30,
             validation_split=0.2,
             batch_size=1024,
             callbacks=callbacks)

Epoch 1/30


2023-05-01 20:54:06.574453: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x59e012d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-01 20:54:06.574474: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2023-05-01 20:54:06.574478: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-05-01 20:54:06.578076: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-01 20:54:06.690376: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


42/42 [==============================] - 30s 543ms/step - loss: 1.7459 - accuracy: 0.3214 - val_loss: 1.6416 - val_accuracy: 0.3747 - lr: 0.0010
Epoch 2/30
42/42 [==============================] - 22s 530ms/step - loss: 1.6379 - accuracy: 0.3746 - val_loss: 1.5913 - val_accuracy: 0.3936 - lr: 0.0010
Epoch 3/30
42/42 [==============================] - 22s 528ms/step - loss: 1.5977 - accuracy: 0.3910 - val_loss: 1.5675 - val_accuracy: 0.4003 - lr: 0.0010
Epoch 4/30
42/42 [==============================] - 22s 531ms/step - loss: 1.5706 - accuracy: 0.4030 - val_loss: 1.5409 - val_accuracy: 0.4157 - lr: 0.0010
Epoch 5/30
42/42 [==============================] - 22s 529ms/step - loss: 1.5443 - accuracy: 0.4137 - val_loss: 1.5100 - val_accuracy: 0.4317 - lr: 0.0010
Epoch 6/30
42/42 [==============================] - 23s 546ms/step - loss: 1.5215 - accuracy: 0.4241 - val_loss: 1.4921 - val_accuracy: 0.4357 - lr: 0.0010
Epoch 7/30
42/42 [==============================] - 23s 549ms/step - loss: 

In [52]:
def best_model_2(hp):
    model = Sequential(name='GloVe-LSTM-DO')
    model.add(Input(shape=(MAX_LEN, 50)))
    
    # Hiperparámetros para la primera capa LSTM
    lstm_units_1 = hp.get("lstm_units_1")
    model.add(LSTM(lstm_units_1, return_sequences=True))
    
    # Hiperparámetros para Dropout
    dropout_rate = hp.get("dropout_rate")
    model.add(Dropout(dropout_rate))
    
    # Hiperparámetros para la segunda capa LSTM
    lstm_units_2 = hp.get("lstm_units_2")
    model.add(LSTM(lstm_units_2))
    
    model.add(Flatten())
    
    # Hiperparámetros para la capa Densa
    dense_units = hp.get("dense_units")
    model.add(Dense(dense_units, activation='relu'))
    
    model.add(Dense(7, activation='softmax'))

    # Hiperparámetros para el optimizador
    learning_rate = hp.get("learning_rate")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [54]:
best_model_gloveLstmDo = best_model_2(best_hp_gloveLstmDo)
best_model_gloveLstmDo.fit(X_train, y_train,
             epochs=30,
             validation_split=0.2,
             batch_size=4093,
             callbacks=callbacks)

Epoch 1/30
11/11 [==============================] - 4s 94ms/step - loss: 1.8823 - accuracy: 0.2309 - val_loss: 1.8294 - val_accuracy: 0.2524 - lr: 8.0808e-04
Epoch 2/30
11/11 [==============================] - 0s 38ms/step - loss: 1.8082 - accuracy: 0.2792 - val_loss: 1.7805 - val_accuracy: 0.3122 - lr: 8.0808e-04
Epoch 3/30
11/11 [==============================] - 0s 38ms/step - loss: 1.7589 - accuracy: 0.3134 - val_loss: 1.7376 - val_accuracy: 0.3338 - lr: 8.0808e-04
Epoch 4/30
11/11 [==============================] - 0s 38ms/step - loss: 1.7116 - accuracy: 0.3513 - val_loss: 1.6981 - val_accuracy: 0.3555 - lr: 8.0808e-04
Epoch 5/30
11/11 [==============================] - 0s 38ms/step - loss: 1.6707 - accuracy: 0.3661 - val_loss: 1.6676 - val_accuracy: 0.3648 - lr: 8.0808e-04
Epoch 6/30
11/11 [==============================] - 0s 38ms/step - loss: 1.6428 - accuracy: 0.3735 - val_loss: 1.6444 - val_accuracy: 0.3733 - lr: 8.0808e-04
Epoch 7/30
11/11 [==============================] - 

In [55]:
def best_model_3(hp):
    model = Sequential(name='3LSTM-DO-CNN-Dense')
    model.add(Input(shape=(MAX_LEN, 50)))

    # Hiperparámetros para la primera capa LSTM
    lstm_units_1 = hp.get("lstm_units_1")
    model.add(LSTM(lstm_units_1, name='LSTM1', return_sequences=True))
    lstm_dropout_1 = hp.get("lstm_dropout_1")
    model.add(Dropout(lstm_dropout_1, name='DO1'))

    # Hiperparámetros para la segunda capa LSTM
    lstm_units_2 = hp.get("lstm_units_2")
    model.add(LSTM(lstm_units_2, name='LSTM2', return_sequences=True))
    lstm_dropout_2 = hp.get("lstm_dropout_2")
    model.add(Dropout(lstm_dropout_2, name='DO2'))

    # Hiperparámetros para la tercera capa LSTM
    lstm_units_3 = hp.get("lstm_units_3")
    model.add(LSTM(lstm_units_3, name='LSTM3', return_sequences=True))
    lstm_dropout_3 = hp.get("lstm_dropout_3")
    model.add(Dropout(lstm_dropout_3, name='DO3'))

    # Hiperparámetros para la capa Conv1D
    conv_filters = hp.get("conv_filters")
    model.add(Conv1D(conv_filters, kernel_size=3, strides=1, padding='same', activation='relu'))
    
    model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))

    cnn_dropout = hp.get("cnn_dropout")
    model.add(Dropout(cnn_dropout))
    
    model.add(Flatten(name='F1'))

    # Hiperparámetros para la capa Densa
    dense_units = hp.get("dense_units")
    model.add(Dense(dense_units, activation='relu'))
    
    model.add(Dense(7, activation='softmax'))

    # Hiperparámetros para el optimizador
    learning_rate = hp.get("learning_rate")
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [61]:
best_model_gloveLstmDoCnn = best_model_3(best_hp_gloveLstmDoCnn)
best_model_gloveLstmDoCnn.fit(X_train, y_train,
             epochs=30,
             validation_split=0.2,
             batch_size=4093,
             callbacks=callbacks)

Epoch 1/30
11/11 [==============================] - 8s 258ms/step - loss: 1.8540 - accuracy: 0.2409 - val_loss: 1.8160 - val_accuracy: 0.2948 - lr: 9.3591e-04
Epoch 2/30
11/11 [==============================] - 2s 181ms/step - loss: 1.7768 - accuracy: 0.2994 - val_loss: 1.7688 - val_accuracy: 0.3132 - lr: 9.3591e-04
Epoch 3/30
11/11 [==============================] - 2s 181ms/step - loss: 1.7234 - accuracy: 0.3353 - val_loss: 1.7023 - val_accuracy: 0.3459 - lr: 9.3591e-04
Epoch 4/30
11/11 [==============================] - 2s 181ms/step - loss: 1.6709 - accuracy: 0.3609 - val_loss: 1.6615 - val_accuracy: 0.3720 - lr: 9.3591e-04
Epoch 5/30
11/11 [==============================] - 2s 182ms/step - loss: 1.6335 - accuracy: 0.3777 - val_loss: 1.6290 - val_accuracy: 0.3815 - lr: 9.3591e-04
Epoch 6/30
11/11 [==============================] - 2s 181ms/step - loss: 1.6008 - accuracy: 0.3877 - val_loss: 1.6007 - val_accuracy: 0.3916 - lr: 9.3591e-04
Epoch 7/30
11/11 [============================

In [62]:
print("Glove LSTM")
score = best_model_gloveLstm.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy:', score[1])

Glove LSTM
413/413 [==============================] - 13s 32ms/step - loss: 1.3057 - accuracy: 0.5175
Test Accuracy: 0.5175026655197144


In [63]:
print("Glove LSTM Do")
score = best_model_gloveLstmDo.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy:', score[1])

Glove LSTM Do
413/413 [==============================] - 1s 2ms/step - loss: 1.4628 - accuracy: 0.4505
Test Accuracy: 0.4505228102207184


In [64]:
print("Glove LSTM Do Cnn")
score = best_model_gloveLstmDoCnn.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy:', score[1])

Glove LSTM Do Cnn
413/413 [==============================] - 1s 3ms/step - loss: 1.4320 - accuracy: 0.4698
Test Accuracy: 0.46976813673973083
